# Feature engineering

In deze notebook leer je de belangrijkste technieken van feature engineering met pandas.
Elke stap bevat uitleg over het doel, nut en verwachte resultaat.

## 📊 1. Dataset laden

We gebruiken een voorbeeld-dataset van een telecomprovider.
Deze kan je vinden op de volgende link: https://www.kaggle.com/datasets/palashfendarkar/wa-fnusec-telcocustomerchurn

Deze dataset bevat gegevens over klanten van een telecombedrijf. Elke rij stelt één klant voor en bevat informatie over:

* Klantkenmerken (zoals geslacht, of ze een partner hebben, of ze afhankelijke personen hebben)
* Abonnementstype (zoals internetdienst, contractduur, betaalmethode)
* Gebruik van diensten (zoals telefonie, tv, cloudopslag, technische ondersteuning)
* Financiële gegevens (zoals maandelijkse en totale uitgaven)
* Churn (of de klant is gestopt met het abonnement of niet)

De dataset is bedoeld voor het voorspellen van churn (klantenverlies), een klassiek classificatieprobleem in machine learning.

Schrijf in de codecellen hieronder de volgende stappen:
* alle benodigde imports
* de code om de dataset te downloaden via kagglehub en in te laden via pandas.
* print de eerste 5 rijen uit om te verifieren of het goed gelukt is.

In [ ]:
# import everything

## Bestuderen dataset

### Globale informatie over de kolommen

En eerste stap om informatie over de beschikbare data te bekomen is door middel van de functie [info()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html).
Deze functie geeft een overzicht van hoeveel rijen en kolommen er in het dataframe zijn, wat voor type data de verschillende kolommen bevatten en hoeveel null-waarden er in elke kolom zitten.
Daarnaast kunnen we met [describe()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) meer informatie opvragen over de verdeling van de numerieke waarden per kolom.

Doel: Begrijpen hoe de dataset eruitziet en voorbereidende stappen uitvoeren.

Waarom: Zo kunnen we foutieve of onbruikbare gegevens aanpakken vóór we nieuwe features maken.

Om een basisidee te hebben over de structuur van de dataset, voer de volgende stappen uit:
* Toon het aantal rijen en kolommen
* Zoek kolommen met ontbrekende waarden
* Verwijder de kolom customerID (uniek ID, geen voorspellende waarde)
* Zet TotalCharges om naar een numeriek datatype zodat we er wiskundige operaties op kunnen uitvoeren.

# 🛠️ 3. Nieuwe features creëren

Doel: Nieuwe kolommen maken die extra informatie toevoegen aan het model.
Waarom: Machine learning-modellen presteren vaak beter met goede, afgeleide features.

Om de dataset beter voor te bereiden op analyse en modellering, voeren we enkele transformaties en verrijkingen uit op de bestaande kolommen. We starten met het afleiden van een binaire indicator voor telefoonservice, gevolgd door het omzetten van contracttypes naar een numerieke duur in maanden. Tot slot berekenen we per klant het totaal aantal geactiveerde optionele diensten, zoals streaming of online back-up. Deze stappen helpen om de gegevens consistenter en modelvriendelijker te maken.

Voer hiervoor de volgende stappen uit:

* Maak een kolom 'has_phone_service' (True/False gebaseerd op PhoneService)
* Maak een kolom 'contract_duration': Month-to-month → 1, One year → 12, Two year → 24
* Tel het totaal aantal actieve optionele diensten per klant (StreamingTV, OnlineBackup...)


Daarnaast werken AI-modellen enkel met numerieke waarden om er berekeningen op uit te kunnen voeren. Categorische variabelen (zoals tekstlabels) moeten dus eerst omgezet worden naar numerieke representaties. 

In deze stap passen we twee vormen van encoding toe:

* Label encoding (of binaire omzetting naar 0/1) gebruiken we voor binaire categorieën zoals gender, Partner en Dependents. Deze kolommen bevatten slechts twee mogelijke waarden (zoals "Male"/"Female" of "Yes"/"No"), waardoor het logisch en efficiënt is om ze te representeren met getallen als 0 en 1.
* One-hot encoding gebruiken we voor kolommen met meer dan twee categorieën, zoals InternetService, Contract en PaymentMethod. Hierbij maken we voor elke unieke categorie een aparte kolom die aangeeft of die categorie van toepassing is (1) of niet (0). Dit voorkomt dat het model een numerieke volgorde veronderstelt die er niet is, zoals het geval zou zijn bij eenvoudige label encoding van meerdere categorieën.

Voer hiervoor de volgende stappen uit: 
* Zet gender, Partner en Dependents om naar 0/1
* Gebruik one-hot encoding voor InternetService, Contract en PaymentMethod

In de vorige stap is categorieke data omgezet in numerieke data. In bepaalde situaties kan het ook handig zijn om niet te werken met de ruwe numerieke data maar deze eerst te verdelen in bepaalde groepen of categorieen. Dit proces wordt ook binning genoemd. Binning is het proces waarbij numerieke waarden worden gegroepeerd in categorieën of klassen. Dit kan helpen om patronen in de data beter zichtbaar te maken of om de invloed van kleine schommelingen (ruis) te verminderen. In sommige gevallen kan het ook bijdragen aan de interpretatie van modellen. In deze stap gebruiken we binning om de kolom MonthlyCharges op twee manieren op te splitsen: eerst in drie vaste categorieën, en daarna met behulp van kwantielen via pd.qcut, zodat elke groep ongeveer evenveel waarnemingen bevat.

Test dit uit door de volgende berekeningen uit te voeren:
* Maak een kolom income_category op basis van MonthlyCharges met 3 bins
* Maak kwantiel-gebaseerde bins met pd.qcut

In de vierde stap voegen we extra tijdsinformatie toe aan de dataset. Datumkolommen kunnen gebruikt worden om afgeleide kenmerken te maken, zoals het jaar, de maand of de dag van de week waarop een klant is begonnen. Dit helpt om patronen te identificeren die verband houden met tijd, zoals seizoensinvloeden of recente klantactiviteit. Indien de dataset geen startdatum bevat, kan optioneel een willekeurige datum worden toegevoegd voor oefendoeleinden. Op basis van deze datum maken we vervolgens enkele afgeleide tijdsvariabelen, waaronder een indicator voor klanten die in 2023 zijn gestart.

Samengevat voer je hieronder dus de volgende stappen uit:
* Omdat het dataframe geen duidelijke startdatum kolom heeft, genereer een kolom met random waarden tussen 2020 en 2025.
* Haal jaar, maand en weekdag uit de datum
* Maak een binaire kolom recent_customer (True als klant in 2023 begon)


Vervolgens gaan we op zoek naar uitschieters in de gegevens — dat zijn waarden die aanzienlijk afwijken van wat gebruikelijk is in een bepaalde kolom. Zulke outliers kunnen het resultaat zijn van meetfouten, uitzonderlijke gevallen of natuurlijke variatie. Omdat ze in sommige modellen een onevenredig grote invloed kunnen hebben op het resultaat, is het belangrijk om ze te identificeren en te overwegen of ze verwijderd moeten worden. We gebruiken de interkwartielafstand (IQR) om grenzen te bepalen voor wat als een uitschieter wordt beschouwd, passen deze methode toe op de MonthlyCharges-kolom, en filteren vervolgens de outliers uit de dataset.

Schrijf hieronder de nodige code om de volgende berekeningen uit te voeren:
* Bereken IQR van MonthlyCharges en bepaal fences
* Zoek outliers volgens deze grenzen
* Verwijder outliers en sla de dataset op

Sommige machine learning-algoritmen, zoals K-Nearest Neighbors (KNN) en Support Vector Machines (SVM), zijn gevoelig voor de schaal van invoervariabelen. Variabelen met grotere waarden kunnen daarbij een onevenredige invloed uitoefenen op het model. Door numerieke features op een consistente schaal te brengen, zorgen we ervoor dat ze gelijkaardig worden gewogen tijdens het leerproces. We gebruiken twee veelvoorkomende technieken: min-max scaling voor MonthlyCharges, waarbij de waarden worden herschaald naar een bereik tussen 0 en 1, en z-score normalisatie voor tenure, waarbij de waarden worden gestandaardiseerd tot een gemiddelde van 0 en standaarddeviatie van 1.

Om dit in te oefenen, voer de volgende schalingen uit:
* Normaliseer MonthlyCharges met min-max scaling
* Standaardiseer tenure naar z-score (gemiddelde 0, std 1)

Een laatste belangrijk onderdeel van feature engineering is door nieuwe kolommen te maken door bestaande kolommen te combineren. Dit maakt het mogelijk om onderliggende verbanden in de dataset te ontdekken die anders niet zichtbaar zijn. Door bijvoorbeeld de maandelijkse kosten te delen door het aantal gebruikte diensten, krijg je een beter beeld van wat een klant gemiddeld betaalt per dienst. Daarnaast kan een samengestelde indicator zoals is_high_value helpen om klanten te identificeren die zowel waardevol zijn als geneigd om te blijven. Deze nieuwe kenmerken kunnen extra informatie bieden aan het model en zo bijdragen aan betere voorspellingen.

Bereken de volgende gecombineerde kolommen om dit in te oefenen:

* monthly_cost_per_service = MonthlyCharges gedeeld door aantal diensten
* is_high_value = True als TotalCharges > 1500 en Churn == "No"
